In [120]:
import pandas as pd
import numpy as np
from pulp import *

import sqlite3

PATH = "C:\\Users\\User\\Desktop\\repos\\QPoland-Hackathon-24\\data\\"

In [121]:
# connection to the database
con = sqlite3.connect(PATH + "smplr_inst.db")

In [122]:
# read as a dataframe
flights = pd.read_sql_query("SELECT * FROM flights", con)
flights.leg_id = flights.leg_id.astype(str)
flights

,index,leg_id,flight_id,date,aircraft,from,to,start,finish,crew_group_id
0,0,70002970,2970,2006-01-07,A320#10,TLS,ORY,2006-01-07T06:40,2006-01-07T08:00,376980210
1,1,70002975,2975,2006-01-07,A320#10,ORY,TLS,2006-01-07T08:50,2006-01-07T10:00,376980210
2,2,70002982,2982,2006-01-07,A320#10,TLS,ORY,2006-01-07T10:40,2006-01-07T12:00,376980210
3,3,70002985,2985,2006-01-07,A320#10,ORY,TLS,2006-01-07T12:50,2006-01-07T14:00,376980210
4,4,70002992,2992,2006-01-07,A320#10,TLS,ORY,2006-01-07T14:40,2006-01-07T16:00,1878748607
5,5,70002999,2999,2006-01-07,A320#10,ORY,TLS,2006-01-07T16:50,2006-01-07T18:00,1878748607
6,6,70003008,3008,2006-01-07,A320#10,TLS,ORY,2006-01-07T18:40,2006-01-07T20:00,1827753950
7,7,70004563,4563,2006-01-07,A321#4,CDG,NCE,2006-01-07T06:20,2006-01-07T07:55,623057530
8,8,70004564,4564,2006-01-07,A321#4,NCE,CDG,2006-01-07T09:05,2006-01-07T10:40,623057530
9,9,70004567,4567,2006-01-07,A321#4,CDG,NCE,2006-01-07T11:35,2006-01-07T13:10,623057530


In [123]:
routes = pd.read_sql_query("SELECT * FROM routes", con)
routes

,index,from,to,type,distance
0,0,CDG,NCE,D,95
1,1,CDG,ORY,D,35
2,2,CDG,TLS,D,85
3,3,NCE,CDG,D,95
4,4,NCE,ORY,D,85
5,5,NCE,TLS,D,85
6,6,ORY,CDG,D,35
7,7,ORY,NCE,D,85
8,8,ORY,TLS,D,80
9,9,TLS,CDG,D,90


In [125]:
# https://github.com/marco-cheung/airport-gate-assignment-problem/tree/main

# create time buckets
# Using discrete time-buckets
min_bucket = 5  # min

# Create time-series between arrival of first plane and departure of last
t = pd.Series(
    pd.date_range(
        start=flights["start"].min(),
        end=flights["finish"].max(),
        freq=pd.offsets.Minute(min_bucket),
    ),
)

A = pd.DataFrame(np.zeros((len(t), len(flights.leg_id))), index=t)
A.columns = flights.leg_id
A

leg_id,70002970,70002975,70002982,70002985,70002992,70002999,70003008,70004563,70004564,70004567,70004568,70004571,70004572
2006-01-07 06:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-01-07 20:55:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
aircraft_allowed_stands = pd.read_excel("ac_types_allowed_stands.xlsx")
aircraft_allowed_stands

,aircraft_code,allowed_stand,code_ref,served_zone
0,388,D201,IATA_code,towing
1,351,D201,IATA_code,towing
2,359,D201,IATA_code,towing
3,346,D201,IATA_code,towing
4,345,D201,IATA_code,towing
...,...,...,...,...
3889,787,S111,AODB_code,towing
3890,777,S111,AODB_code,towing
3891,748,S111,AODB_code,towing
3892,747,S111,AODB_code,towing


In [173]:
aircraft = pd.read_sql_query("SELECT * FROM aircraft", con)
aircraft

,index,registration,model,family,position,owned
0,0,A321#4,A321,Airbus,CDG,1
1,1,A320#10,A320,Airbus,TLS,1


In [308]:
# 0. Initialise model
lp = LpProblem("Flight_Scheduler", LpMaximize)  # minimize cost

In [309]:
# # 1. Define Variable: x[i,j] = (0,1)
# # Binary = leg_i allocated to aircraft_j
# x = {}
# for leg in A.columns:
#     for airplane in aircraft[aircraft.owned == 1].registration.values:
#         x[leg, airplane] = LpVariable("{}_{}".format(leg, airplane), 0, 1, LpBinary)

# 1. Define Variable: x[i,j] = (0,inf)
# Binary = time_i is allocated to leg_j
x = {}
for leg in A.columns:
    for time in A.index:
        x[time, leg] = LpVariable(f"x_{time}_{leg}", lowBound=0, cat=LpInteger)

# Non zero indicators
y = {}
for leg in A.columns:
    for time in A.index:
        y[time, leg] = LpVariable(f"y_{time}_{leg}", cat=LpBinary)

In [310]:
# 2. Set the objective function as 0 for now (ignore 0 for now)
lp += A.sum().sum(), "lost"

In [311]:
# 3. Constraints


# i. Each leg must be assigned to only one time

# for leg in A.columns:
#     lp += lpSum(x[leg, t] for t in A.index if (leg, t) in x) == 1

for leg in A.columns:
    lp += lpSum(y[time, leg] for time in A.index) == 1, f"One_nonzero_in_column_{leg}"

# dependence bewtwween x an y
M = 10_000  # constrain value for x[i, j]
for leg in A.columns:
    for time in A.index:
        lp += x[time, leg] <= M * y[time, leg], f"Link_x_y_{time}_{leg}"

# ii. Each time must have no more assigned then the number of available planes
for t in A.index:
    lp += (
        lpSum(y[time, leg] for leg in A.columns)
        <= aircraft[aircraft.owned == 1].shape[0]
    )

In [312]:
# 4. Solve
lp.solve()

1

In [313]:
# 5. check the status, variables and objective function
LpStatus[lp.status]

'Optimal'

In [314]:
results = A.copy()
for leg in A.columns:
    for time in A.index:
        results.loc[time, leg] = value(x[time, leg])
results

leg_id,70002970,70002975,70002982,70002985,70002992,70002999,70003008,70004563,70004564,70004567,70004568,70004571,70004572
2006-01-07 06:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-01-07 20:55:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [317]:
results.sum().sum()

np.float64(0.0)